In [1]:
from ontobio.config import set_config
set_config('conf/config.yaml')



/Users/cjm/repos/mondo-build/src/ontology/venv/lib/python3.6/site-packages/cachier/mongo_core.py:24: UserWarning: Cachier warning: pymongo was not found. MongoDB cores will not work.
  "Cachier warning: pymongo was not found. MongoDB cores will not work.")


In [2]:
from ontobio.ontol_factory import OntologyFactory

ofactory = OntologyFactory()
mondo = ofactory.create("mondo.json") ## interpreted as a json local handle

In [3]:
go = ofactory.create("go") ## remote
asmap = {}

In [5]:
go

h:go g:

In [7]:
from ontobio import AssociationSetFactory
afactory = AssociationSetFactory()

taxa = ['NCBITaxon:4896', 'NCBITaxon:4932', 'NCBITaxon:7955', 'NCBITaxon:10090', 'NCBITaxon:9606']

for taxon in taxa:
    print("LOOKUP: {}".format(taxon))
    if taxon in asmap:
        print("CACHED")
        continue
    aset = afactory.create(ontology=go, subject_category='gene', object_category='function', taxon=taxon)
    asmap[taxon] = aset

LOOKUP: NCBITaxon:4896
CACHED
LOOKUP: NCBITaxon:4932
CACHED
LOOKUP: NCBITaxon:7955
CACHED
LOOKUP: NCBITaxon:10090
CACHED
LOOKUP: NCBITaxon:9606


In [8]:
from ontobio.golr.golr_associations import bulk_fetch

from ontobio.vocabulary.relations import HomologyTypes

#results = bulk_fetch(subject_category='gene', object_category='gene', taxon='NCBITaxon:9606', 
#               relation=HomologyTypes.Ortholog.value)


In [9]:
[d] = mondo.search('Alzheimers disease')

In [10]:
import pandas as pd


In [11]:
df = pd.read_csv('mim2gene_medgen', sep='\t')


In [12]:
df[0:5]

,#MIM number,GeneID,type,Source,MedGenCUI,Comment
0,100050,-,phenotype,-,C3149220,-
1,100070,-,phenotype,-,C0162871,-
2,100100,1131,phenotype,GeneMap,C0033770,question
3,100200,-,phenotype,-,C0271355,-
4,100300,57514,phenotype,GeneMap,CN028867,-


In [13]:
df = df.loc[df['type'] == 'phenotype']

In [14]:
df[0:10]

,#MIM number,GeneID,type,Source,MedGenCUI,Comment
0,100050,-,phenotype,-,C3149220,-
1,100070,-,phenotype,-,C0162871,-
2,100100,1131,phenotype,GeneMap,C0033770,question
3,100200,-,phenotype,-,C0271355,-
4,100300,57514,phenotype,GeneMap,CN028867,-
5,100600,-,phenotype,-,C0000889,-
10,100675,-,phenotype,-,C1863486,-
13,100700,-,phenotype,-,C1332135,-
20,100800,2261,phenotype,GeneMap,C0001080,-
21,100820,-,phenotype,-,C1863416,-


In [15]:
eg = mondo.equiv_graph()

In [16]:
tups = []
notfound = []
for ix,row in df.iterrows():
    gid = row['GeneID']
    if gid == '-':
        continue
    if row['Comment'] == 'nondisease':
        continue
    mim = row['#MIM number']
    mim_uri = "http://purl.obolibrary.org/obo/OMIM_{}".format(mim)
    if mim_uri not in eg:
        #print("NOT FOUND:{} {}".format(mim, row['Source']))
        notfound.append(row)
        continue
    for d in eg.neighbors(mim_uri):
        tups.append( (d, mondo.label(d), 'NCBIGene:{}'.format(gid)) )

len(tups)

5623

In [17]:
        
notfound[0:20]

[#MIM number       110800
 GeneID              2651
 type           phenotype
 Source           GeneMap
 MedGenCUI       C1862229
 Comment                -
 Name: 440, dtype: object, #MIM number       111150
 GeneID             10661
 type           phenotype
 Source           GeneMap
 MedGenCUI       C1292231
 Comment                -
 Name: 445, dtype: object, #MIM number       111400
 GeneID             53947
 type           phenotype
 Source           GeneMap
 MedGenCUI       C0599990
 Comment                -
 Name: 451, dtype: object, #MIM number       113750
 GeneID            283652
 type           phenotype
 Source           GeneMap
 MedGenCUI       C2676042
 Comment                -
 Name: 528, dtype: object, #MIM number               138900
 GeneID                      9429
 type                   phenotype
 Source                   GeneMap
 MedGenCUI               C1841837
 Comment        nondisease; QTL 2
 Name: 1711, dtype: object, #MIM number       141749
 GeneID        

In [20]:
tups[0:20]

[('http://purl.obolibrary.org/obo/MONDO_0007032',
  'prune belly syndome',
  'NCBIGene:1131'),
 ('http://purl.obolibrary.org/obo/MONDO_0007034',
  'Adams-Oliver syndrome',
  'NCBIGene:57514'),
 ('http://purl.obolibrary.org/obo/MONDO_0007037',
  'achondroplasia',
  'NCBIGene:2261'),
 ('http://purl.obolibrary.org/obo/MONDO_0007039',
  'neurofibromatosis type 2',
  'NCBIGene:4771'),
 ('http://purl.obolibrary.org/obo/MONDO_0007041',
  'Apert syndrome',
  'NCBIGene:2263'),
 ('http://purl.obolibrary.org/obo/MONDO_0007042',
  'Saethre-Chotzen syndrome',
  'NCBIGene:2263'),
 ('http://purl.obolibrary.org/obo/MONDO_0007042',
  'Saethre-Chotzen syndrome',
  'NCBIGene:7291'),
 ('http://purl.obolibrary.org/obo/MONDO_0007043',
  'Pfeiffer syndrome',
  'NCBIGene:2260'),
 ('http://purl.obolibrary.org/obo/MONDO_0007043',
  'Pfeiffer syndrome',
  'NCBIGene:2263'),
 ('http://purl.obolibrary.org/obo/MONDO_0007044',
  'acrodysostosis 1 with or without hormone resistance',
  'NCBIGene:5573'),
 ('http://purl

In [21]:
godict = {}

In [22]:
from ontobio.golr.golr_query import GolrAssociationQuery
msolr = "https://solr.monarchinitiative.org/solr/golr"
golr = "http://golr.berkeleybop.org"
from ontobio.vocabulary.relations import HomologyTypes

def orthogo(gid):
    if gid in godict:
        return godict[gid]
    terms = set()
    hq = GolrAssociationQuery(subject_category='gene', object_category='gene', subject=gid, 
                              relation=HomologyTypes.Ortholog.value,
                              rows=100,
                              url=msolr)
    #print("PARAMS={}".format(hq.solr_params()))
    assocs = hq.exec()
    for a in assocs['associations']:
        orth = a['object']['id']
        #print("LOOKUP: {}".format(orth))
        for taxon, aset in asmap.items():
            #hq = GolrAssociationQuery(object_category='function', subject=orth, url=golr, rows=1000)
            #for fa in hq.exec()['associations']:
            for ann in aset.annotations(orth):
                terms.add(str(ann))
    godict[gid] = terms
    return terms

terms = orthogo('NCBIGene:5313')

In [23]:
terms

{'GO:0000166',
 'GO:0000287',
 'GO:0001889',
 'GO:0003824',
 'GO:0004743',
 'GO:0005524',
 'GO:0005737',
 'GO:0005829',
 'GO:0005975',
 'GO:0006096',
 'GO:0006754',
 'GO:0008152',
 'GO:0009408',
 'GO:0010226',
 'GO:0016301',
 'GO:0016310',
 'GO:0016740',
 'GO:0030955',
 'GO:0032869',
 'GO:0033198',
 'GO:0042866',
 'GO:0046872',
 'GO:0051591',
 'GO:0051707',
 'GO:0070062'}

In [24]:
m2go = {}
n = 0
for t in tups:
    terms = orthogo(t[2])
    mid = t[0]
    if mid not in m2go:
        m2go[mid] = set()
    m2go[mid].update(terms)
    n += 1
    if n % 100 == 1:
        print("{} -> {}".format(mid, terms))
    

http://purl.obolibrary.org/obo/MONDO_0007032 -> {'GO:0003056', 'GO:0004871', 'GO:0045987', 'GO:0016907', 'GO:0007271', 'GO:0045211', 'GO:0007207', 'GO:0007165', 'GO:0005887', 'GO:0004930', 'GO:0007197', 'GO:0005886', 'GO:0030054', 'GO:0016021', 'GO:0007186', 'GO:0016020', 'GO:0046541', 'GO:0045202'}
http://purl.obolibrary.org/obo/MONDO_0007204 -> {'GO:0004656', 'GO:0031545'}
http://purl.obolibrary.org/obo/MONDO_0007311 -> {'GO:0010977', 'GO:0030154', 'GO:0003674', 'GO:0008150', 'GO:0007050', 'GO:0043218', 'GO:0032060', 'GO:0032288', 'GO:0042552', 'GO:0008219', 'GO:0008285', 'GO:0005886', 'GO:0007049', 'GO:0016021', 'GO:0016020', 'GO:0005923'}
http://purl.obolibrary.org/obo/MONDO_0007455 -> {'GO:0003674', 'GO:0031018', 'GO:0055074', 'GO:1990798', 'GO:0005783', 'GO:0000502', 'GO:0042593', 'GO:0016021', 'GO:0016020', 'GO:0003323'}
http://purl.obolibrary.org/obo/MONDO_0007631 -> set()
http://purl.obolibrary.org/obo/MONDO_0007784 -> {'GO:0043565', 'GO:0004887', 'GO:0043010', 'GO:0006351', '

http://purl.obolibrary.org/obo/MONDO_0011122 -> {'GO:0007218', 'GO:0005576', 'GO:0043473', 'GO:2000253', 'GO:0070062', 'GO:0005794', 'GO:0070996', 'GO:0005615', 'GO:0043025', 'GO:0048589', 'GO:0005796', 'GO:0005102', 'GO:0009755', 'GO:0060259', 'GO:0031781', 'GO:0040008', 'GO:0008343', 'GO:0005184', 'GO:0031782', 'GO:0042755'}
http://purl.obolibrary.org/obo/MONDO_0011285 -> {'GO:0071711', 'GO:0033334', 'GO:0005509', 'GO:0033333'}
http://purl.obolibrary.org/obo/MONDO_0011439 -> {'GO:0005856', 'GO:0004722'}
http://purl.obolibrary.org/obo/MONDO_0011585 -> {'GO:0006810', 'GO:0006629', 'GO:0042995', 'GO:0009611', 'GO:0005637', 'GO:0005789', 'GO:0030054', 'GO:0005515', 'GO:0014069', 'GO:0008202', 'GO:0005783', 'GO:0043523', 'GO:0007399', 'GO:0045211', 'GO:0016021', 'GO:0004872', 'GO:0000247', 'GO:0042802', 'GO:0005640', 'GO:0070207', 'GO:0006696', 'GO:0006869', 'GO:0005635', 'GO:0005886', 'GO:0005811', 'GO:0031410', 'GO:0016126', 'GO:0005634', 'GO:0016853', 'GO:0004985', 'GO:0016020', 'GO:00

http://purl.obolibrary.org/obo/MONDO_0014592 -> {'GO:0051011'}
http://purl.obolibrary.org/obo/MONDO_0014694 -> {'GO:0045892', 'GO:0001757', 'GO:0006351', 'GO:0032525', 'GO:0060349', 'GO:0007219', 'GO:0009880', 'GO:0005634', 'GO:0000122', 'GO:0007368', 'GO:0036342', 'GO:0010468', 'GO:0007275', 'GO:0006355', 'GO:0009798', 'GO:0048641', 'GO:0005515', 'GO:0001503', 'GO:0001756'}
http://purl.obolibrary.org/obo/MONDO_0014800 -> {'GO:0044297', 'GO:0030017', 'GO:0005737', 'GO:0005884', 'GO:0048741', 'GO:0043234', 'GO:0005856', 'GO:0005515', 'GO:0072562', 'GO:0005524', 'GO:0030027', 'GO:0000166', 'GO:0090131', 'GO:0070062', 'GO:0030175', 'GO:0001725', 'GO:0015629', 'GO:0005865', 'GO:0005615', 'GO:0010628', 'GO:0035050', 'GO:0030240'}
http://purl.obolibrary.org/obo/MONDO_0014902 -> {'GO:0045087', 'GO:0004672', 'GO:0004713', 'GO:0021549', 'GO:0033339', 'GO:0001889', 'GO:0051897', 'GO:0016301', 'GO:0021587', 'GO:0021522', 'GO:0016310', 'GO:0031017', 'GO:0005524', 'GO:0001764', 'GO:0021707', 'GO:00

In [25]:
[apert] = mondo.search('Apert syndrome')
m2go[apert]

{'GO:0000166',
 'GO:0003140',
 'GO:0004672',
 'GO:0004713',
 'GO:0004714',
 'GO:0005007',
 'GO:0005515',
 'GO:0005524',
 'GO:0005794',
 'GO:0005886',
 'GO:0006468',
 'GO:0006915',
 'GO:0008284',
 'GO:0008543',
 'GO:0016020',
 'GO:0016021',
 'GO:0016301',
 'GO:0016310',
 'GO:0016740',
 'GO:0017134',
 'GO:0030901',
 'GO:0031012',
 'GO:0031410',
 'GO:0033278',
 'GO:0035469',
 'GO:0048565',
 'GO:0048701',
 'GO:0060271',
 'GO:0061371',
 'GO:0071910'}

In [26]:
apert

'http://purl.obolibrary.org/obo/MONDO_0007041'

In [30]:
rows = []
for d,terms in m2go.items():
    for t in terms:
        rows.append( {'disease': d, 'disease_name': mondo.label(d), 'go':t, 'go_name': go.label(t)} )

df = pd.DataFrame(rows)
df.set_index('disease')
df.to_csv('mondo-leaves-to-go.tsv', sep='\t')
df[0:5]

,disease,disease_name,go,go_name
0,http://purl.obolibrary.org/obo/MONDO_0007032,prune belly syndome,GO:0003056,regulation of vascular smooth muscle contraction
1,http://purl.obolibrary.org/obo/MONDO_0007032,prune belly syndome,GO:0004871,signal transducer activity
2,http://purl.obolibrary.org/obo/MONDO_0007032,prune belly syndome,GO:0045987,positive regulation of smooth muscle contraction
3,http://purl.obolibrary.org/obo/MONDO_0007032,prune belly syndome,GO:0007271,"synaptic transmission, cholinergic"
4,http://purl.obolibrary.org/obo/MONDO_0007032,prune belly syndome,GO:0007207,phospholipase C-activating G-protein coupled a...


In [81]:
mtups = []
for d,terms in m2go.items():
    for t in terms:
        mtups.append( (d, mondo.label(d), t) )
        
mset = afactory.create_from_tuples(mtups, ontology=go)

In [ ]:
msimple = mondo.subontology(relations='subClassOf')

In [89]:
eobjs = []
for d in msimple.nodes():
    if not d.startswith('http://purl.obolibrary.org/obo/MONDO'):
        continue
    leafs = msimple.descendants(d)
    if len(leafs) < 2:
        continue
    enr = mset.enrichment_test(subjects=leafs, threshold=0.000005, labels=True)
    if len(enr) == 0:
        continue
    #print('{} {}'.format(d, msimple.label(d)))
    for r in enr:
       #print('  {}'.format(r))
        r['disease'] = d
        r['disease label'] = mondo.label(d)
        eobjs.append(r)
        if len(eobjs) % 1000 == 1:
            print('{}'.format(r))



{'c': 'GO:0006767', 'p': 3.3528193217327691e-09, 'p_uncorrected': 2.6317263121921265e-12, 'n': 'water-soluble vitamin metabolic process', 'disease': 'http://purl.obolibrary.org/obo/MONDO_0001639', 'disease label': 'deficiency anemia'}
{'c': 'GO:0050863', 'p': 4.5076412611943789e-08, 'p_uncorrected': 9.042409751643689e-12, 'n': 'regulation of T cell activation', 'disease': 'http://purl.obolibrary.org/obo/MONDO_0015823', 'disease label': 'primary immunodeficiency due to a defect in adaptive immunity'}
{'c': 'GO:0007265', 'p': 1.2375779678481437e-07, 'p_uncorrected': 7.9535859116204605e-11, 'n': 'Ras protein signal transduction', 'disease': 'http://purl.obolibrary.org/obo/MONDO_0021060', 'disease label': 'RASopathy'}
{'c': 'GO:0035082', 'p': 2.7407890616283519e-28, 'p_uncorrected': 2.4691793348003171e-31, 'n': 'axoneme assembly', 'disease': 'http://purl.obolibrary.org/obo/MONDO_0018395', 'disease label': 'male infertility due to sperm motility disorder'}
{'c': 'GO:0050664', 'p': 3.0763569

In [90]:
edf = pd.DataFrame(eobjs)

In [99]:
edf[3000:4999]

,c,disease,disease label,n,p,p_uncorrected
3000,GO:0035082,http://purl.obolibrary.org/obo/MONDO_0018395,male infertility due to sperm motility disorder,axoneme assembly,2.740789e-28,2.469179e-31
3001,GO:0097711,http://purl.obolibrary.org/obo/MONDO_0018395,male infertility due to sperm motility disorder,ciliary basal body-plasma membrane docking,2.740789e-28,2.469179e-31
3002,GO:0042073,http://purl.obolibrary.org/obo/MONDO_0018395,male infertility due to sperm motility disorder,intraciliary transport,5.141390e-28,4.631883e-31
3003,GO:0048199,http://purl.obolibrary.org/obo/MONDO_0018395,male infertility due to sperm motility disorder,"vesicle targeting, to, from or within Golgi",7.006935e-28,6.312554e-31
3004,GO:0001578,http://purl.obolibrary.org/obo/MONDO_0018395,male infertility due to sperm motility disorder,microtubule bundle formation,8.170072e-28,7.360425e-31
3005,GO:0044782,http://purl.obolibrary.org/obo/MONDO_0018395,male infertility due to sperm motility disorder,cilium organization,8.170072e-28,7.360425e-31
3006,GO:0061512,http://purl.obolibrary.org/obo/MONDO_0018395,male infertility due to sperm motility disorder,protein localization to cilium,9.518704e-28,8.575409e-31
3007,GO:0006903,http://purl.obolibrary.org/obo/MONDO_0018395,male infertility due to sperm motility disorder,vesicle targeting,1.108122e-27,9.983077e-31
3008,GO:0098840,http://purl.obolibrary.org/obo/MONDO_0018395,male infertility due to sperm motility disorder,protein transport along microtubule,1.498280e-27,1.349802e-30
3009,GO:0099118,http://purl.obolibrary.org/obo/MONDO_0018395,male infertility due to sperm motility disorder,microtubule-based protein transport,1.498280e-27,1.349802e-30


In [98]:
len(edf)

11379

In [100]:
edf.to_csv('mondo-to-go-enr.tsv', sep='\t')

In [102]:
edf = edf[['disease', 'disease label', 'c', 'n', 'p']]

In [106]:
df = edf.set_index('disease')

In [104]:
edf.to_csv('mondo-to-go-enr.tsv', sep='\t')

In [107]:
df[0:5]

,disease label,c,n,p
disease,,,,
http://purl.obolibrary.org/obo/MONDO_0001639,deficiency anemia,GO:0006767,water-soluble vitamin metabolic process,3.352819e-09
http://purl.obolibrary.org/obo/MONDO_0001639,deficiency anemia,GO:0009235,cobalamin metabolic process,6.864751e-09
http://purl.obolibrary.org/obo/MONDO_0001639,deficiency anemia,GO:0031419,cobalamin binding,2.047286e-08
http://purl.obolibrary.org/obo/MONDO_0001639,deficiency anemia,GO:0019842,vitamin binding,2.959151e-06
http://purl.obolibrary.org/obo/MONDO_0000359,spondylocostal dysostosis,GO:0001756,somitogenesis,3.349297e-07


In [108]:
df.to_csv('mondo-to-go-enr.tsv', sep='\t')